# Cosmological Power Spectra Calculation
This code calculates and stores cosmological power spectra 
and related coefficients for weak lensing and matter power spectra based on cosmological models.

In [1]:
import os
import json
import numpy
import pyccl
import scipy

In [ ]:
# FOLDER
FOLDER = '/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

Choosing a cosmological model.

In [ ]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY_INFO = json.load(file)

COSMOLOGY = pyccl.Cosmology(
    h = COSMOLOGY_INFO['H'],
    w0 = COSMOLOGY_INFO['W0'],
    wa = COSMOLOGY_INFO['WA'], 
    n_s = COSMOLOGY_INFO['NS'], 
    A_s = COSMOLOGY_INFO['AS'],
    m_nu = COSMOLOGY_INFO['M_NU'],  
    Neff = COSMOLOGY_INFO['N_EFF'],
    Omega_b = COSMOLOGY_INFO['OMEGA_B'], 
    Omega_k = COSMOLOGY_INFO['OMEGA_K'], 
    Omega_c = COSMOLOGY_INFO['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

Calculate the power spectrum.

In [ ]:
# Define the redshift grid
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

# Calculate corresponding scale factors and comoving distances
A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

# Calculate the power spectrum on the grid
POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=COSMOLOGY, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

Compute the numerical integral needed for the covariance matrix.

In [5]:
# Numerical integral

def integral_I5(chi1, chi2, chi3, power1, power2, redshift1, redshift2):
    
    def formula_0(chi):
        
        result = (chi2 - chi) / (chi * (chi2 - chi1))
        
        result = result * ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2 ** 2 - 2 * chi1 * chi2 + chi ** 2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))
        
        result = result * (((chi/chi2) ** 3)*power2)
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)
        
        return result
    
    def formula_n(chi):
        result = (chi2 - chi) / (chi * (chi2 - chi1))
        
        result = result * ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2 ** 2 - 2 * chi1 * chi2 + chi ** 2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

Calculate the coefficients that are used in the angular power spectrum computation.

In [6]:
# Coefficient

def coefficient_J5(chi1, chi2, chi3, power1, power2, redshift1, redshift2):
    a = 1 - chi1 / chi2
    b = chi3 / chi2 - 1
    p = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)
    
    if a == 1:
        formula = (7 * b * (7 + 5 * b) - 2 * b * (9 + 7 * b) * z + 14 * (1 + b) * ( - 3 + z) * numpy.log(1 + b)) / (840 * b)
    else:
        formula = - ((1 / (720 * a ** 4 * b)) * (a * b * (10 * a * (6 * a * ( - 12 + 5 * a ** 2 + 6 * a * (2 + b)) + 60 * p - a * (66 + 28 * a + 17 * a ** 2 + 18 * (2 + a) * b) * p) + 
( - 10 * a * ( - 60 + a * (66 + 28 * a + 17 * a ** 2 + 18 * (2 + a) * b)) + 3 * ( - 180 + a * (30 * (7 + 4 * b) + a * (90 + 60 * b + a * (55 + 39 * a + 40 * b)))) * p) * z) + 
60 * (b * (6 * a ** 3 * (3 + b) + 10 * a * p - 9 * p * z + a * (10 + 15 * p + 6 * b * p) * z - 2 * a ** 2 * (6 + 8 * p + 3 * b * p + 8 * z + 3 * b * z) + a ** 5 * ( - 6 + 4 * p + 4 * z - 3 * p * z) + 
a ** 4 * (6 - 4 * p - 4 * z + 3 * p * z)) * numpy.log(1 - a) + a ** 5 * (1 + b) * (6 - 4 * z + p * ( - 4 + 3 * z)) * numpy.log(1 + b))))
    
    coefficient = chi2 * power2 * (1 + redshift2) * formula
    return coefficient

In [7]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = numpy.float128(CHI_GRID[GRID_INDEX])
CHI2 = numpy.float128(CHI_GRID[GRID_INDEX + 1])
CHI3 = numpy.float128(CHI_GRID[GRID_INDEX + 2])

POWER1 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX])
POWER2 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX + 1])

REDSHIFT1 = numpy.float128(Z_GRID[GRID_INDEX])
REDSHIFT2 = numpy.float128(Z_GRID[GRID_INDEX + 1])

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

4924.221744025123371 4940.716645699534638 4957.1305889213954288 11370.147631619063759 11268.969970366384587 1.75 1.7600000000000000089


Compute the integral and coefficient for the case where chi1 > 0

In [8]:
# Case1: chi1 > 0

INTEGRAL = integral_I5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

1.9554263194108604481 1.9554263173632202314 -1.0471579503577681247e-09


In [9]:
# Case2: chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.40057919571794 88.58943766284152 0.0 1826.8838600584127 0.0 0.01


Compute the integral and coefficient for the case where chi1 = 0

In [10]:
# Case2: chi1 = 0

INTEGRAL = integral_I5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

2491.524698743292 2491.5246987432915 -2.220446049250313e-16
